<a href="https://colab.research.google.com/github/yellowjs0304/3-min-pytorch/blob/master/04-%ED%8C%A8%EC%85%98_%EC%95%84%EC%9D%B4%ED%85%9C%EC%9D%84_%EA%B5%AC%EB%B6%84%ED%95%98%EB%8A%94_DNN/neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 뉴럴넷으로 패션 아이템 구분하기
Fashion MNIST 데이터셋과 앞서 배운 인공신경망을 이용하여 패션아이템을 구분해봅니다.

In [ ]:
import torch
# DNN 모델의 재료라 할 수 있는 nn 모듈
import torch.nn as nn
# optimizer를 위한 optim 모듈
import torch.optim as optim
# nn 모듈의 함수 버전 functional 모듈
import torch.nn.functional as F
# torchvision의 데이터셋인 MNIST를 사용하려 아래 라이브러리 임포트
from torchvision import transforms, datasets

In [ ]:
# 현재 컴퓨터에서 CUDA(GPU Library)를 이용할 수 있는지 알아보기 위한 Flag
USE_CUDA = torch.cuda.is_available()
# USE_CUDA가 True일 경우, GPU를 할당 그 외엔 CPU 메모리 할당
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [ ]:
# Epoch과 Batch_size 선언
EPOCHS = 30
BATCH_SIZE = 64

## 데이터셋 불러오기

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor()
])

In [ ]:
trainset = datasets.FashionMNIST(
    root      = './.data/', 
    train     = True,
    download  = True,
    transform = transform
)
testset = datasets.FashionMNIST(
    root      = './.data/', 
    train     = False,
    download  = True,
    transform = transform
)

train_loader = torch.utils.data.DataLoader(
    dataset     = trainset,
    batch_size  = BATCH_SIZE,
    shuffle     = True,
)
test_loader = torch.utils.data.DataLoader(
    dataset     = testset,
    batch_size  = BATCH_SIZE,
    shuffle     = True,
)

## 뉴럴넷으로 Fashion MNIST 학습하기

입력 `x` 는 `[배치크기, 색, 높이, 넓이]`로 이루어져 있습니다.
`x.size()`를 해보면 `[64, 1, 28, 28]`이라고 표시되는 것을 보실 수 있습니다.
Fashion MNIST에서 이미지의 크기는 28 x 28, 색은 흑백으로 1 가지 입니다.
그러므로 입력 x의 총 특성값 갯수는 28 x 28 x 1, 즉 784개 입니다.

우리가 사용할 모델은 3개의 레이어를 가진 인공신경망 입니다. 

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        # input data(image) = x 
        # x.shape = torch.Size([64, 1, 28, 28])
        import pdb; pdb.set_trace()
        x = x.view(-1, 784)
        # x.shape = torch.Size([64, 784])
        x = F.relu(self.fc1(x))
        # x.shape = torch.Size([64, 256])
        x = F.relu(self.fc2(x))
        # x.shape = torch.Size([64, 128])
        x = self.fc3(x)
        # x.shape = torch.Size([64, 10])
        return x

## 모델 준비하기

`to()` 함수는 모델의 파라미터들을 지정한 곳으로 보내는 역할을 합니다.
일반적으로 CPU 1개만 사용할 경우 필요는 없지만,
GPU를 사용하고자 하는 경우 `to("cuda")`로 지정하여 GPU로 보내야 합니다.
지정하지 않을 경우 계속 CPU에 남아 있게 되며 빠른 훈련의 이점을 누리실 수 없습니다.

최적화 알고리즘으로 파이토치에 내장되어 있는 `optim.SGD`를 사용하겠습니다.

In [ ]:
# GPU 사용을 위함
model        = Net().to(DEVICE)

#최적화 함수 SGD
optimizer    = optim.SGD(model.parameters(), lr=0.01)

## 학습하기

In [ ]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # 학습 데이터를 DEVICE의 메모리로 보냄
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

## 테스트하기

In [ ]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 모든 오차 더하기
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()
            
            # 가장 큰 값을 가진 클래스가 모델의 예측입니다.
            # 예측과 정답을 비교하여 일치할 경우 correct에 1을 더합니다.
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

## 코드 돌려보기

자, 이제 모든 준비가 끝났습니다. 코드를 돌려서 실제로 훈련이 되는지 확인해봅시다!

In [ ]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

> <ipython-input-16-eb1d98073a48>(10)forward()
-> x = x.view(-1, 784)
(Pdb) x.shape
torch.Size([64, 1, 28, 28])
(Pdb) n
> <ipython-input-16-eb1d98073a48>(11)forward()
-> x = F.relu(self.fc1(x))
(Pdb) x.shape
torch.Size([64, 784])
(Pdb) b = self.fc1(x)
*** The specified object '= self.fc1(x)' is not a function or was not found along sys.path.
(Pdb) line
*** NameError: name 'line' is not defined
(Pdb) b = self.fc1(x)
*** The specified object '= self.fc1(x)' is not a function or was not found along sys.path.
(Pdb) bb=self.fc1(x)
(Pdb) bb.shape
torch.Size([64, 256])
(Pdb) n
> <ipython-input-16-eb1d98073a48>(12)forward()
-> x = F.relu(self.fc2(x))
(Pdb) x.shape
torch.Size([64, 256])
(Pdb) n
> <ipython-input-16-eb1d98073a48>(13)forward()
-> x = self.fc3(x)
(Pdb) x.shape
torch.Size([64, 128])
(Pdb) n
> <ipython-input-16-eb1d98073a48>(14)forward()
-> return x
(Pdb) x.shape
torch.Size([64, 10])
(Pdb) ㅌ
*** NameError: name 'ᄐ' is not defined
(Pdb) x
tensor([[ 7.9308e-02,  3.2331e-03, -5.9811e-02